In [320]:
import pandas as pd
import numpy as np
from statsmodels.distributions.empirical_distribution import ECDF
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [10, 5]
import seaborn as sns
from scipy import stats
import math

**Исследование зависимостей: Номинальные признаки.**

**Критерий хи-квадрат Пирсона.**

In [321]:
def criteria_chisquared_pearson(table, a=0.05):
    value_stats_criteria = 0.0
    for i in range(table.shape[0] - 1):
        for j in range(table.shape[1] - 1):
            value_stats_criteria += (table[i,j] - table[i,-1] * table[-1,j] / table[-1,-1])**(2) / (table[i,-1] * table[-1,j] / table[-1,-1])
  
    chi2 = stats.chi2.ppf(1 - a, (table.shape[0] - 2) * (table.shape[1] - 2))

    print("Значение статистики критерия -", value_stats_criteria, "\nКритическое значение критерия хи-квадрат Пирсона -", chi2)

    pvalue = 1 - stats.norm.cdf(value_stats_criteria)
    print("P-значение - {}".format(pvalue))

    if value_stats_criteria <= chi2:
        print("Гипотеза H0 принимется (признаки независимы)")
    else: print("Гипотеза H0 отвергается в пользу Ha (признаки зависимы)")

    return value_stats_criteria

In [322]:
def link_coefficients(criteria_chisquared_pearson, total_value, count_rows, count_cols, table):
    coef1 = math.sqrt(criteria_chisquared_pearson / total_value)
    print("{} - {}".format("Коэффициент среднеквадратической сопряжённости", coef1))
    coef2 = math.sqrt(criteria_chisquared_pearson / total_value)
    print("{} - {}".format("Коэффициент Пирсона", coef2))
    coef3 = math.sqrt(criteria_chisquared_pearson / (total_value * math.sqrt((count_rows - 1) * (count_cols - 1))))
    print("{} - {}".format("Коэффициент Чупрова", coef3))
    coef4 = math.sqrt(criteria_chisquared_pearson / (total_value * min((count_rows - 1), (count_cols - 1))))
    print("{} - {}".format("Коэффициент Крамера", coef4))

    Pb1 = 1 - np.max(table[-1, :-1]) / total_value
    Pb2 = 1 - np.sum(np.max(table[:-1, :-1], axis=1)) / total_value
    Pa1 = 1 - np.max(table[:-1, -1]) / total_value
    Pa2 = 1 - np.sum(np.max(table[:-1, :-1], axis=0)) / total_value

    Lb = (Pb1 - Pb2) / Pb1
    La = (Pa1 - Pa2) / Pa1

    print("Меры прогноза Гутмана: La - {}, Lb - {}".format(La, Lb))

    if count_rows == 2 and count_cols == 2:
        a = table[0,0]; b = table[0,1]; c = table[1,0]; d = table[1,1]

        coef5 = (a*d - b*c) / math.sqrt((a + b) * (c + d) * (a + c) * (b + d))
        print("{} - {}".format("Коэффициент контингенции", coef5))

        coef6 = (a*d - b*c) / (a*d + b*c)
        print("{} - {}".format("Коэффициент ассоциации", coef6))

**Задача 1.1.**

In [323]:
table = np.array([
    [12, 48, 47],
    [20, 478, 666],
    [11, 160, 701]
])

In [324]:
table = table.astype(np.float32)

table = np.concatenate((table, np.sum(table, axis = 0)[np.newaxis, :]), axis=0)
table = np.concatenate((table, np.sum(table, axis = 1)[:, np.newaxis]), axis=1)

table

array([[  12.,   48.,   47.,  107.],
       [  20.,  478.,  666., 1164.],
       [  11.,  160.,  701.,  872.],
       [  43.,  686., 1414., 2143.]], dtype=float32)

In [325]:
val = criteria_chisquared_pearson(table, 0.05)
print("")
link_coefficients(val, table[-1,-1], table.shape[0] - 1, table.shape[1] - 1, table)

Значение статистики критерия - 183.16555698660235 
Критическое значение критерия хи-квадрат Пирсона - 9.487729036781154
P-значение - 0.0
Гипотеза H0 отвергается в пользу Ha (признаки зависимы)

Коэффициент среднеквадратической сопряжённости - 0.2923551980433383
Коэффициент Пирсона - 0.2923551980433383
Коэффициент Чупрова - 0.20672634305158058
Коэффициент Крамера - 0.20672634305158058
Меры прогноза Гутмана: La - 0.035750818063681106, Lb - 0.0013715940558183413


***Выводы: Признаки имеют зависимость ниже средней.***

**Задача 1.2.**

In [326]:
table = np.array([
    [168, 92],
    [85, 135]
])

In [327]:
table = table.astype(np.float32)

table = np.concatenate((table, np.sum(table, axis = 0)[np.newaxis, :]), axis=0)
table = np.concatenate((table, np.sum(table, axis = 1)[:, np.newaxis]), axis=1)

In [328]:
val = criteria_chisquared_pearson(table, 0.05)
print("")
link_coefficients(val, table[-1,-1], table.shape[0] - 1, table.shape[1] - 1, table)

Значение статистики критерия - 32.265356570881565 
Критическое значение критерия хи-квадрат Пирсона - 3.841458820694124
P-значение - 0.0
Гипотеза H0 отвергается в пользу Ha (признаки зависимы)

Коэффициент среднеквадратической сопряжённости - 0.25926722287247045
Коэффициент Пирсона - 0.25926722287247045
Коэффициент Чупрова - 0.25926722287247045
Коэффициент Крамера - 0.25926722287247045
Меры прогноза Гутмана: La - 0.19545456259703908, Lb - 0.22026436104345415
Коэффициент контингенции - 0.25926721756930693
Коэффициент ассоциации - 0.4872131049633026


***Выводы: Признаки имеют зависимость ниже средней.***

**Исследование зависимостей: Порядковые и количественные признаки.**

In [329]:
def get_ranges(sequence):
    maxvalue = 10000000000
    ranks = [0] * len(sequence)

    temp_sequence = list(sequence.copy())

    rank = 1
    while True:
        minval = min(temp_sequence)
        idx = temp_sequence.index(minval)

        if minval == maxvalue: break

        ranks[idx] = rank
        temp_sequence[idx] = maxvalue
        rank += 1

    return np.array(ranks)

In [330]:
def criteria_coef_correlation(X, Y, a=0.05):
    r = (np.sum((X - np.mean(X)) * (Y - np.mean(Y))) / len(X)) / np.sqrt((np.sum((X - np.mean(X))**2) / len(X)) * (np.sum((Y - np.mean(Y))**2) / len(Y)))

    print("Коэффициент корреляции -", r)

    value_stats_criteria = (r * math.sqrt(len(X) - 2)) / math.sqrt(1 - r**2)

    tmin = stats.t.ppf(a / 2, len(X) - 2)
    tmax = stats.t.ppf(1 - a / 2, len(X) - 2)

    print("Значение статистики критерия -", value_stats_criteria) 
    print("Квантиль стандартного нормального распределения: уровня альфа/2 - {}, уровня 1 - альфа/2 - {}".format(tmin, tmax))

    pvalue = 2 * stats.norm.cdf(value_stats_criteria)
    pvalue2 = 2 - 2 * stats.norm.cdf(value_stats_criteria)
    if pvalue > pvalue2: pvalue = pvalue2

    if value_stats_criteria > tmin and value_stats_criteria < tmax: return ("H0 (coef of correlation = 0) - accept", "PValue={}".format(pvalue))

    return ("H0 (coef of correlation = 0) - reject", "PValue={}".format(pvalue))

    return value_stats_criteria

In [331]:
def criteria_spearman(X, Y, a=0.05):
    n = len(X)

    r = get_ranges(X)
    s = get_ranges(Y)

    S = np.sum((r - s)**2)

    ro = 1 - (6 * S / (n * n**2 - n))

    print("Ранговый коэффициент корреляции Спирмена -", ro)

    value_stats_criteria = math.sqrt(n - 1) * ro

    tmin = stats.t.ppf(a / 2, 1000)
    tmax = stats.t.ppf(1 - a / 2, 1000)

    print("Значение статистики критерия -", value_stats_criteria) 
    print("Квантиль стандартного нормального распределения: уровня альфа/2 - {}, уровня 1 - альфа/2 - {}".format(tmin, tmax))

    pvalue = 2 * stats.norm.cdf(value_stats_criteria)
    pvalue2 = 2 - 2 * stats.norm.cdf(value_stats_criteria)
    if pvalue > pvalue2: pvalue = pvalue2

    if value_stats_criteria > tmin and value_stats_criteria < tmax: return ("H0 (coef of correlation = 0) - accept", "PValue={}".format(pvalue))

    return ("H0 (coef of correlation = 0) - reject", "PValue={}".format(pvalue))

    return value_stats_criteria

In [332]:
def criteria_kendall(X, Y, a=0.05):
    n = len(X)

    r_ = list(get_ranges(X))
    r = r_[:]
    r.sort()  # Сортируем по возрасанию
    s_ = list(get_ranges(Y))
    s = [0] * n
  
    for idx in range(n):
        val = r_[idx]
        idx_ = r.index(val)
        s[idx_] = s_[idx]

    K = 0  # число несогласованных пар
    for idx in range(n - 1):
        for idx_ in range(idx + 1, n):
            if s[idx_] < s[idx]: K += 1

    t = 1 - (4 * K / (n * (n - 1)))

    print("Коэффициент согласованности (коэффициент Кендалла) -", t)

    value_stats_criteria = (3 * math.sqrt(n) / 2) * t

    tmin = stats.t.ppf(a / 2, 1000)
    tmax = stats.t.ppf(1 - a / 2, 1000)

    print("Значение статистики критерия -", value_stats_criteria) 
    print("Квантиль стандартного нормального распределения: уровня альфа/2 - {}, уровня 1 - альфа/2 - {}".format(tmin, tmax))

    pvalue = 2 * stats.norm.cdf(value_stats_criteria)
    pvalue2 = 2 - 2 * stats.norm.cdf(value_stats_criteria)
    if pvalue > pvalue2: pvalue = pvalue2

    if value_stats_criteria > tmin and value_stats_criteria < tmax: return ("H0 (coef of correlation = 0) - accept", "PValue={}".format(pvalue))

    return ("H0 (coef of correlation = 0) - reject", "PValue={}".format(pvalue))

    return value_stats_criteria

In [333]:
def transfer_to_table(X, Y, N):
    x_step = (np.max(X) - np.min(X)) / N
    y_step = (np.max(Y) - np.min(Y)) / N

    table = []
    for y in np.arange(np.min(Y), np.max(Y), y_step):
        row = []
        for x in np.arange(np.min(X), np.max(X), x_step):
            count = 0
            for i in range(len(X)):
                if (x <= X[i] < x + x_step) and (y <= Y[i] < y + y_step): count += 1
            row.append(count)
        table.append(row)
    return np.array(table)

**Задача 2.1**

In [334]:
X = np.array([22.49, 22.56, 23.45, 22.58, 24.3, 24.2, 23.47, 23.5, 24.48, 25.02, 23.04, 23.24, 25.2, 24.61, 26.02])
Y = np.array([52.93, 53.4, 53.7, 53.36, 61.8, 55.2, 53.54, 58.33, 60.4, 60.3, 54.28, 53.6, 62.24, 54.45, 61.52])

In [335]:
criteria_coef_correlation(X, Y)

Коэффициент корреляции - 0.807916997741104
Значение статистики критерия - 4.943160478465096
Квантиль стандартного нормального распределения: уровня альфа/2 - -2.160368656461013, уровня 1 - альфа/2 - 2.1603686564610127


('H0 (coef of correlation = 0) - reject', 'PValue=7.686615934865415e-07')

In [336]:
criteria_spearman(X, Y)

Ранговый коэффициент корреляции Спирмена - 0.8821428571428571
Значение статистики критерия - 3.300676337618441
Квантиль стандартного нормального распределения: уровня альфа/2 - -1.9623390808264078, уровня 1 - альфа/2 - 1.9623390808264074


('H0 (coef of correlation = 0) - reject', 'PValue=0.0009645208069288813')

In [337]:
criteria_kendall(X, Y)

Коэффициент согласованности (коэффициент Кендалла) - 0.6952380952380952
Значение статистики критерия - 4.038968346759163
Квантиль стандартного нормального распределения: уровня альфа/2 - -1.9623390808264078, уровня 1 - альфа/2 - 1.9623390808264074


('H0 (coef of correlation = 0) - reject', 'PValue=5.3686808705766254e-05')

In [338]:
table = transfer_to_table(X, Y, 2)
table = np.concatenate((table, np.sum(table, axis = 0)[np.newaxis, :]), axis=0)
table = np.concatenate((table, np.sum(table, axis = 1)[:, np.newaxis]), axis=1)
table

array([[ 8,  1,  9],
       [ 1,  3,  4],
       [ 9,  4, 13]])

In [339]:
val = criteria_chisquared_pearson(table, 0.05)
print("")
link_coefficients(val, table[-1,-1], table.shape[0] - 1, table.shape[1] - 1, table)

Значение статистики критерия - 5.306327160493827 
Критическое значение критерия хи-квадрат Пирсона - 3.841458820694124
P-значение - 5.592806096021974e-08
Гипотеза H0 отвергается в пользу Ha (признаки зависимы)

Коэффициент среднеквадратической сопряжённости - 0.6388888888888888
Коэффициент Пирсона - 0.6388888888888888
Коэффициент Чупрова - 0.6388888888888888
Коэффициент Крамера - 0.6388888888888888
Меры прогноза Гутмана: La - 0.5, Lb - 0.5
Коэффициент контингенции - 0.6388888888888888
Коэффициент ассоциации - 0.92


***Выводы: Признаки имеют высокую зависимость.***

**Задача 2.2**

In [340]:
X = np.array([20.1, 23.6, 26.3, 19.9, 16.7, 23.2, 31.4, 33.5, 28.2, 35.3, 29.3])
Y = np.array([7.2, 7.1, 7.4, 6.1, 6.0, 7.3, 9.4, 9.2, 8.8, 10.4, 8.0])

In [341]:
criteria_coef_correlation(X, Y)

Коэффициент корреляции - 0.9485888243336862
Значение статистики критерия - 8.99104946703741
Квантиль стандартного нормального распределения: уровня альфа/2 - -2.262157162740992, уровня 1 - альфа/2 - 2.2621571627409915


('H0 (coef of correlation = 0) - reject', 'PValue=0.0')

In [342]:
criteria_spearman(X, Y)

Ранговый коэффициент корреляции Спирмена - 0.9545454545454546
Значение статистики критерия - 3.0185377665243625
Квантиль стандартного нормального распределения: уровня альфа/2 - -1.9623390808264078, уровня 1 - альфа/2 - 1.9623390808264074


('H0 (coef of correlation = 0) - reject', 'PValue=0.002539977411205818')

In [343]:
criteria_kendall(X, Y)

Коэффициент согласованности (коэффициент Кендалла) - 0.8545454545454545
Значение статистики критерия - 4.251309958546467
Квантиль стандартного нормального распределения: уровня альфа/2 - -1.9623390808264078, уровня 1 - альфа/2 - 1.9623390808264074


('H0 (coef of correlation = 0) - reject', 'PValue=2.1252380079328503e-05')

In [344]:
table = transfer_to_table(X, Y, 2)

table = np.concatenate((table, np.sum(table, axis = 0)[np.newaxis, :]), axis=0)
table = np.concatenate((table, np.sum(table, axis = 1)[:, np.newaxis]), axis=1)
table

array([[ 5,  2,  7],
       [ 0,  3,  3],
       [ 5,  5, 10]])

In [345]:
val = criteria_chisquared_pearson(table, 0.05)
print("")
link_coefficients(val, table[-1,-1], table.shape[0] - 1, table.shape[1] - 1, table)

Значение статистики критерия - 4.285714285714286 
Критическое значение критерия хи-квадрат Пирсона - 3.841458820694124
P-значение - 9.10764857453561e-06
Гипотеза H0 отвергается в пользу Ha (признаки зависимы)

Коэффициент среднеквадратической сопряжённости - 0.6546536707079771
Коэффициент Пирсона - 0.6546536707079771
Коэффициент Чупрова - 0.6546536707079771
Коэффициент Крамера - 0.6546536707079771
Меры прогноза Гутмана: La - 0.3333333333333336, Lb - 0.6000000000000001
Коэффициент контингенции - 0.6546536707079772
Коэффициент ассоциации - 1.0


***Выводы: Признаки имеют высокую зависимость.***